In [1]:
#Import necessary functions
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from datetime import datetime as dt

%matplotlib inline

In [2]:
#Import the dataset and merge Crash Date and Crash Time columns
df = pd.read_csv('data/Motor_Vehicle_Collisions_-_Crashes.csv', low_memory=False, parse_dates=[['CRASH DATE', 'CRASH TIME']])

#Initial look into dataset
df.head()

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1979921 entries, 0 to 1979920
Data columns (total 28 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   CRASH DATE_CRASH TIME          datetime64[ns]
 1   BOROUGH                        object        
 2   ZIP CODE                       object        
 3   LATITUDE                       float64       
 4   LONGITUDE                      float64       
 5   LOCATION                       object        
 6   ON STREET NAME                 object        
 7   CROSS STREET NAME              object        
 8   OFF STREET NAME                object        
 9   NUMBER OF PERSONS INJURED      float64       
 10  NUMBER OF PERSONS KILLED       float64       
 11  NUMBER OF PEDESTRIANS INJURED  int64         
 12  NUMBER OF PEDESTRIANS KILLED   int64         
 13  NUMBER OF CYCLIST INJURED      int64         
 14  NUMBER OF CYCLIST KILLED       int64         
 15  NUMBER OF MOTOR

In [4]:
#Check what the dates range from
df['CRASH DATE_CRASH TIME'].describe()

/var/folders/v0/p588qkrd5kb8kdnf_3zrlbm80000gn/T/ipykernel_22963/3699960668.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df['CRASH DATE_CRASH TIME'].describe()


count                 1979921
unique                1067748
top       2015-01-18 08:00:00
freq                       51
first     2012-07-01 00:05:00
last      2023-03-27 23:46:00
Name: CRASH DATE_CRASH TIME, dtype: object

## Data Preparation

### Changing DateTime Index

In [5]:
#Rename crash date column
df.rename(columns = {'CRASH DATE_CRASH TIME':'CRASH DATE TIME'}, inplace = True)

#Set to datetime index
df.set_index('CRASH DATE TIME', inplace=True)

The collisions beyond 2023 will be considered future data. It will not be considered within the modeling process.

In [6]:
#Create dataframe with only 2023 data - to be considered future
#df_2023 = df['2023':]

The collisions within the last full 5 years will be considered in the model only.

In [7]:
#Filter to crashes between 2018-2023
df = df['2018-01-01':]

In [8]:
#Sanity check of the modeling data time frame
print (df.index.min())
print (df.index.max())

2018-01-01 00:00:00
2023-03-27 23:46:00


In [9]:
#Sort the datetime indexes in ascending order
df = df.sort_index()

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 791077 entries, 2018-01-01 00:00:00 to 2023-03-27 23:46:00
Data columns (total 27 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   BOROUGH                        514688 non-null  object 
 1   ZIP CODE                       514567 non-null  object 
 2   LATITUDE                       730480 non-null  float64
 3   LONGITUDE                      730480 non-null  float64
 4   LOCATION                       730480 non-null  object 
 5   ON STREET NAME                 591371 non-null  object 
 6   CROSS STREET NAME              382154 non-null  object 
 7   OFF STREET NAME                198962 non-null  object 
 8   NUMBER OF PERSONS INJURED      791071 non-null  float64
 9   NUMBER OF PERSONS KILLED       791063 non-null  float64
 10  NUMBER OF PEDESTRIANS INJURED  791077 non-null  int64  
 11  NUMBER OF PEDESTRIANS KILLED   791077 non-null  int64  
 

In [11]:
#Check for any duplicated rows
df.duplicated().value_counts()

False    791077
dtype: int64

### Borough Selection

The two of the five NYC boroughs with the largest populations will be analyzed: Queens and Brooklyn.

In [12]:
#Filter for collisions in Brooklyn and Queens borough
df = df.loc[(df['BOROUGH'] == 'BROOKLYN') | (df['BOROUGH'] == 'QUEENS')]

In [13]:
#Sanity check to ensure only two boroughs are included
print(df['BOROUGH'].unique())

['QUEENS' 'BROOKLYN']


### Dropping Columns/Values

In [14]:
#Check for NaN values in street name columns
print(df['ON STREET NAME'].isnull().sum())
print(df['CROSS STREET NAME'].isnull().sum())
print(df['OFF STREET NAME'].isnull().sum())

115219
115422
199029


There are 6 columns dedicated to where the collision took place. The 'LOCATION' column is to be dropped as it contains repetitive information from the 'LATITUDE' and 'LONGITUDE' columns. The columns regarding street name will be dropped as well due to the significant amount of missing values. 

There are a total of 8 columns dedicated to the number of people injured and killed as a result of the collision. The columns with the total number of injured and killed, whereas the columns specifying the type of person will be dropped. 

In [15]:
print(df['LOCATION'].isnull().sum())

7033


In [16]:
#Drop non-relevant and repetitive location columns
df.drop(['LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME'], axis=1, inplace=True)

#Drop repetitive injury/death columns
df.drop(['NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 
         'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED'], 
        axis=1, inplace=True)

The focus will be on the Vehicle 1 as it is to be considered the primary vehicle of the collision. The columns for the contributing factor and vehicle type code for vehicles 2-5 have a significant amount of missing values and therefore will be dropped from the dataset.

In [17]:
#Drop additional contributing factor vehicle columns
df.drop(['CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 
         'CONTRIBUTING FACTOR VEHICLE 5'], axis=1, inplace=True)

#Drop additional vehicle type columns
df.drop(['VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'], 
        axis=1, inplace=True)


In [18]:
#Drop NaN values from columns of contributing factors
df = df.dropna(subset=['CONTRIBUTING FACTOR VEHICLE 1'])

In [19]:
#Filter out the collisions with Unspecified contributing factor
df = df[df['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']
df

,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,CONTRIBUTING FACTOR VEHICLE 1,COLLISION_ID,VEHICLE TYPE CODE 1
CRASH DATE TIME,,,,,,,,,
2018-01-01 00:00:00,QUEENS,11419,40.682970,-73.828240,0.0,0.0,Passing Too Closely,3819067,Sedan
2018-01-01 00:00:00,BROOKLYN,11239,40.653060,-73.882020,0.0,0.0,Driver Inattention/Distraction,3820851,Sedan
2018-01-01 00:00:00,BROOKLYN,11230,40.624180,-73.970480,0.0,0.0,Driver Inattention/Distraction,3822296,Station Wagon/Sport Utility Vehicle
2018-01-01 00:00:00,BROOKLYN,11234,40.628770,-73.918250,1.0,0.0,Failure to Yield Right-of-Way,3820945,Sedan
2018-01-01 00:00:00,BROOKLYN,11230,40.623220,-73.961020,0.0,0.0,Driver Inattention/Distraction,3821055,Taxi
...,...,...,...,...,...,...,...,...,...
2023-03-27 21:50:00,QUEENS,11385,40.709038,-73.898460,1.0,0.0,Failure to Yield Right-of-Way,4616419,Moped
2023-03-27 22:10:00,BROOKLYN,11218,40.640410,-73.969124,1.0,0.0,Driver Inattention/Distraction,4616379,Sedan
2023-03-27 23:07:00,BROOKLYN,11218,40.640316,-73.979070,0.0,0.0,Backing Unsafely,4616380,Box Truck


#### Number of Persons Injured & Number of Persons Killed

In [20]:
#Check for NaN values in injury/death columns
print(df['NUMBER OF PERSONS KILLED'].isnull().sum())
print(df['NUMBER OF PERSONS INJURED'].isnull().sum())

1
0


Since there were only 2 collisions with missing values from the 'NUMBER OF PERSONS KILLED' column and only 3 from the 'NUMBER OF PERSONS INJURED', these rows will be dropped.  

In [21]:
#Drop NaN values from injury/death columns
df = df.dropna(subset=['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED'])

#Sanity check to verify no more NaN values
print(df['NUMBER OF PERSONS KILLED'].isnull().sum())
print(df['NUMBER OF PERSONS INJURED'].isnull().sum())

0
0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 227935 entries, 2018-01-01 00:00:00 to 2023-03-27 23:46:00
Data columns (total 9 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   BOROUGH                        227935 non-null  object 
 1   ZIP CODE                       227859 non-null  object 
 2   LATITUDE                       222873 non-null  float64
 3   LONGITUDE                      222873 non-null  float64
 4   NUMBER OF PERSONS INJURED      227935 non-null  float64
 5   NUMBER OF PERSONS KILLED       227935 non-null  float64
 6   CONTRIBUTING FACTOR VEHICLE 1  227935 non-null  object 
 7   COLLISION_ID                   227935 non-null  int64  
 8   VEHICLE TYPE CODE 1            226352 non-null  object 
dtypes: float64(4), int64(1), object(4)
memory usage: 17.4+ MB


#### Latitude and Longitude
The Latitude and Longitude columns will be kept to ensure the location of the collisions can be mapped using Folium. First, we must verify if there are any missing values.

In [23]:
#Check for NaN values in Latitude and Longitude columns
print(df['LATITUDE'].isnull().sum())
print(df['LONGITUDE'].isnull().sum())

5062
5062


Since there are less than 5,000 rows missing values for Latitude and Longitude, these rows will be dropped.

In [24]:
#Drop missing values from Latitude and Longitude columns
df = df.dropna(subset=['LATITUDE', 'LONGITUDE'])

Now that we know there are no duplicate 

In [25]:
#Check the minimum/maximum values of Latitude
df['LATITUDE'].describe()

count    222873.000000
mean         40.466272
std           2.996388
min           0.000000
25%          40.656980
50%          40.688150
75%          40.723885
max          40.896500
Name: LATITUDE, dtype: float64

In [26]:
#Check the minimum/maximum values of Longitude
df['LONGITUDE'].describe()

count    222873.000000
mean        -73.490381
std           5.441501
min         -74.194600
25%         -73.951200
50%         -73.907370
75%         -73.833460
max           0.000000
Name: LONGITUDE, dtype: float64

In [27]:
#Check how many collisions have a Latitude of 0
df[df['LATITUDE'] == 0]

,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,CONTRIBUTING FACTOR VEHICLE 1,COLLISION_ID,VEHICLE TYPE CODE 1
CRASH DATE TIME,,,,,,,,,
2018-02-14 14:23:00,BROOKLYN,11219,0.0,0.0,0.0,0.0,Failure to Yield Right-of-Way,4025794,Station Wagon/Sport Utility Vehicle
2018-02-23 11:00:00,QUEENS,11374,0.0,0.0,0.0,0.0,Failure to Yield Right-of-Way,3851213,Station Wagon/Sport Utility Vehicle
2018-02-28 12:00:00,BROOKLYN,11217,0.0,0.0,0.0,0.0,Alcohol Involvement,3853952,Van
2018-02-28 12:30:00,QUEENS,11385,0.0,0.0,0.0,0.0,Oversized Vehicle,3855414,TRAIL
2018-02-28 12:44:00,BROOKLYN,11201,0.0,0.0,0.0,0.0,Oversized Vehicle,3853954,Box Truck
...,...,...,...,...,...,...,...,...,...
2023-03-02 12:00:00,BROOKLYN,11203,0.0,0.0,0.0,0.0,Driver Inattention/Distraction,4611376,Station Wagon/Sport Utility Vehicle
2023-03-09 16:02:00,BROOKLYN,11211,0.0,0.0,0.0,0.0,Passing Too Closely,4612778,Sedan
2023-03-17 15:40:00,BROOKLYN,11236,0.0,0.0,1.0,0.0,Driver Inattention/Distraction,4615443,E-Bike


In [28]:
#Check how many collisions have a Longitude of 0
df[df['LONGITUDE'] == 0]

,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,CONTRIBUTING FACTOR VEHICLE 1,COLLISION_ID,VEHICLE TYPE CODE 1
CRASH DATE TIME,,,,,,,,,
2018-02-14 14:23:00,BROOKLYN,11219,0.0,0.0,0.0,0.0,Failure to Yield Right-of-Way,4025794,Station Wagon/Sport Utility Vehicle
2018-02-23 11:00:00,QUEENS,11374,0.0,0.0,0.0,0.0,Failure to Yield Right-of-Way,3851213,Station Wagon/Sport Utility Vehicle
2018-02-28 12:00:00,BROOKLYN,11217,0.0,0.0,0.0,0.0,Alcohol Involvement,3853952,Van
2018-02-28 12:30:00,QUEENS,11385,0.0,0.0,0.0,0.0,Oversized Vehicle,3855414,TRAIL
2018-02-28 12:44:00,BROOKLYN,11201,0.0,0.0,0.0,0.0,Oversized Vehicle,3853954,Box Truck
...,...,...,...,...,...,...,...,...,...
2023-03-02 12:00:00,BROOKLYN,11203,0.0,0.0,0.0,0.0,Driver Inattention/Distraction,4611376,Station Wagon/Sport Utility Vehicle
2023-03-09 16:02:00,BROOKLYN,11211,0.0,0.0,0.0,0.0,Passing Too Closely,4612778,Sedan
2023-03-17 15:40:00,BROOKLYN,11236,0.0,0.0,1.0,0.0,Driver Inattention/Distraction,4615443,E-Bike


There are 161 rows with both Longitude and Latitude of 0. Looking at the head and tail, we can assume these's are all the same 161 rows. To ensure our data is well-rounded with complete and accurate values, let's drop the collisions with a Longitude and Latitude of 0.

In [29]:
#Filter for collisions that do NOT have Longitude and Latitude of 0
df = df[df['LONGITUDE'] != 0]
df

,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,CONTRIBUTING FACTOR VEHICLE 1,COLLISION_ID,VEHICLE TYPE CODE 1
CRASH DATE TIME,,,,,,,,,
2018-01-01 00:00:00,QUEENS,11419,40.682970,-73.828240,0.0,0.0,Passing Too Closely,3819067,Sedan
2018-01-01 00:00:00,BROOKLYN,11239,40.653060,-73.882020,0.0,0.0,Driver Inattention/Distraction,3820851,Sedan
2018-01-01 00:00:00,BROOKLYN,11230,40.624180,-73.970480,0.0,0.0,Driver Inattention/Distraction,3822296,Station Wagon/Sport Utility Vehicle
2018-01-01 00:00:00,BROOKLYN,11234,40.628770,-73.918250,1.0,0.0,Failure to Yield Right-of-Way,3820945,Sedan
2018-01-01 00:00:00,BROOKLYN,11230,40.623220,-73.961020,0.0,0.0,Driver Inattention/Distraction,3821055,Taxi
...,...,...,...,...,...,...,...,...,...
2023-03-27 21:50:00,QUEENS,11385,40.709038,-73.898460,1.0,0.0,Failure to Yield Right-of-Way,4616419,Moped
2023-03-27 22:10:00,BROOKLYN,11218,40.640410,-73.969124,1.0,0.0,Driver Inattention/Distraction,4616379,Sedan
2023-03-27 23:07:00,BROOKLYN,11218,40.640316,-73.979070,0.0,0.0,Backing Unsafely,4616380,Box Truck


In [30]:
#Sanity check how many collisions have a Longitude of 0
df[df['LONGITUDE'] == 0]

,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,CONTRIBUTING FACTOR VEHICLE 1,COLLISION_ID,VEHICLE TYPE CODE 1
CRASH DATE TIME,,,,,,,,,


In [31]:
#Sanity check how many collisions have a Latitude of 0
df[df['LATITUDE'] == 0]

,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,CONTRIBUTING FACTOR VEHICLE 1,COLLISION_ID,VEHICLE TYPE CODE 1
CRASH DATE TIME,,,,,,,,,


### Imputation

In [32]:
#Check rows with missing zip code
df[df['ZIP CODE'].isna()]

,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,CONTRIBUTING FACTOR VEHICLE 1,COLLISION_ID,VEHICLE TYPE CODE 1
CRASH DATE TIME,,,,,,,,,
2018-01-17 20:49:00,QUEENS,NaN,40.72010,-73.79038,0.0,0.0,Passing or Lane Usage Improper,3830150,Sedan
2018-01-18 17:20:00,QUEENS,NaN,40.72010,-73.79038,0.0,0.0,Driver Inattention/Distraction,3831700,Sedan
2018-02-01 09:00:00,QUEENS,NaN,40.72010,-73.79038,0.0,0.0,Following Too Closely,3839658,Sedan
2018-02-09 17:22:00,QUEENS,NaN,40.72010,-73.79038,0.0,0.0,Following Too Closely,3843234,Bus
2018-03-19 21:00:00,QUEENS,NaN,40.76092,-73.82680,0.0,0.0,Following Too Closely,3865703,Sedan
...,...,...,...,...,...,...,...,...,...
2022-06-27 09:27:00,QUEENS,NaN,40.72013,-73.79038,1.0,0.0,Unsafe Lane Changing,4541523,Station Wagon/Sport Utility Vehicle
2022-08-09 12:30:00,QUEENS,NaN,40.75089,-73.93663,0.0,0.0,Driver Inattention/Distraction,4554698,Station Wagon/Sport Utility Vehicle
2022-11-22 20:08:00,QUEENS,NaN,40.72013,-73.79038,4.0,0.0,Driver Inattention/Distraction,4584239,Station Wagon/Sport Utility Vehicle


There are 71 missing values from the Zip Code column, however, we see that numerous rows have the same latitude and longitude. Let's impute those rows instead of dropping them.

In [33]:
#Create a dictionary that corresponds the Latitude, Longitude with the correct Zip Code
zipcode_dict = {(40.72010, -73.79038): 11432,
                (40.76092, -73.82680): 11354,
                (40.72013, -73.79038): 11433,
                (40.75089, -73.93663): 11101,
                (40.724792, -73.722916): 11426,
                (40.713050, -73.916990): 11385,
                (40.711930, -73.919365): 11385,
                (40.724792, -73.722916): 11426,
                (40.707447, -73.903870): 11385,
                (40.707485, -73.918365): 11385,
                (40.733120, -73.727900): 11426,
                (40.719124, -73.791405): 11432, 
                (40.606260, -73.744170): 11691,
                (40.707317, -73.903595): 11385,
                (40.695072, -73.990100): 11201,}

# use the map function to update the 'zipcode' column based on the dictionary
df['ZIP CODE'] = df.apply(lambda row: zipcode_dict.get((row['LATITUDE'], row['LONGITUDE']), row['ZIP CODE']), axis=1)

In [34]:
#Sanity check to confirm there's no more NaN values in Zip Code column
df[df['ZIP CODE'].isna()]

,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,CONTRIBUTING FACTOR VEHICLE 1,COLLISION_ID,VEHICLE TYPE CODE 1
CRASH DATE TIME,,,,,,,,,


In [35]:
#Sanity check to confirm zipcode was imputed for indexed row
df[df.index.isin(['2018-01-17 20:49:00'])]

,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,CONTRIBUTING FACTOR VEHICLE 1,COLLISION_ID,VEHICLE TYPE CODE 1
CRASH DATE TIME,,,,,,,,,
2018-01-17 20:49:00,QUEENS,11432,40.7201,-73.79038,0.0,0.0,Passing or Lane Usage Improper,3830150,Sedan


### Feature Engineering

#### Contributing Categories

In [36]:
#Review the contributing factors
df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts(sort=True).head()

Driver Inattention/Distraction    76614
Failure to Yield Right-of-Way     26472
Following Too Closely             16305
Backing Unsafely                  15547
Passing Too Closely               13110
Name: CONTRIBUTING FACTOR VEHICLE 1, dtype: int64

In [37]:
#Check the unique values of contributing factors for vehicle 1
df['CONTRIBUTING FACTOR VEHICLE 1'].unique()

array(['Passing Too Closely', 'Driver Inattention/Distraction',
       'Failure to Yield Right-of-Way', 'Unsafe Speed',
       'Traffic Control Device Improper/Non-Working',
       'Traffic Control Disregarded', 'Pavement Slippery',
       'Following Too Closely', 'Backing Unsafely',
       'Passenger Distraction', 'Passing or Lane Usage Improper',
       'Other Vehicular', 'Alcohol Involvement', 'Driver Inexperience',
       'Fell Asleep', 'Brakes Defective', 'View Obstructed/Limited',
       'Unsafe Lane Changing', 'Obstruction/Debris', 'Turning Improperly',
       'Pavement Defective', 'Reaction to Uninvolved Vehicle',
       'Aggressive Driving/Road Rage', 'Tire Failure/Inadequate',
       'Steering Failure',
       'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
       'Animals Action', 'Glare', 'Oversized Vehicle',
       'Outside Car Distraction', 'Failure to Keep Right',
       'Other Electronic Device', 'Lost Consciousness',
       'Driverless/Runaway Vehicle', 'Other

In [38]:
#Create duplicate columns of contributing factor columns for categorizing
df['CONTRIBUTING CATEGORY V1'] = df['CONTRIBUTING FACTOR VEHICLE 1']

#Replace value with correct spelling of 'Illness'
df['CONTRIBUTING FACTOR VEHICLE 1'].replace('Illnes', 'Illness', inplace=True)

In [39]:
#Replace specific contributing factor to a more generalized category in Contributing Category Vehicle 1
df['CONTRIBUTING CATEGORY V1'].replace(('Driver Inattention/Distraction', 'Driver Inexperience', 
                                        'Reaction to Uninvolved Vehicle', 'Aggressive Driving/Road Rage', 
                                        'Eating or Drinking'), ('Driver Error'), inplace=True)

df['CONTRIBUTING CATEGORY V1'].replace(('Tire Failure/Inadequate', 'Headlights Defective', 'Steering Failure', 
                                        'Brakes Defective', 'Accelerator Defective', 'Tow Hitch Defective', 
                                        'Other Lighting Defects', 'Tinted Windows', 'Vehicle Vandalism', 
                                        'Windshield Inadequate'), ('Vehicle Defects'), inplace=True)

df['CONTRIBUTING CATEGORY V1'].replace(('Failure to Yield Right-of-Way','Passing or Lane Usage Improper', 
                                        'Unsafe Lane Changing','Failure to Keep Right', 
                                        'Traffic Control Disregarded','Passing Too Closely', 
                                        'Backing Unsafely', 'Unsafe Speed', 'Following Too Closely', 
                                        'Turning Improperly'), ('Moving Violation'), inplace=True)

df['CONTRIBUTING CATEGORY V1'].replace(('Glare', 'Obstruction/Debris', 'View Obstructed/Limited'), 
                                       ('Environmental Factors'), inplace=True)

df['CONTRIBUTING CATEGORY V1'].replace(('Cell Phone (hand-Held)', 'Texting', 'Using On Board Navigation Device', 
                                        'Other Electronic Device', 'Listening/Using Headphones', 
                                        'Cell Phone (hands-free)'), ('Internal Electronics Usage'), inplace=True)

df['CONTRIBUTING CATEGORY V1'].replace(('Illnes', 'Illness', 'Drugs (illegal)', 'Fell Asleep', 'Fatigued/Drowsy', 
                                        'Lost Consciousness', 'Physical Disability', 'Alcohol Involvement', 
                                        'Prescription Medication'), ('Bodily Impairment'), inplace=True)

df['CONTRIBUTING CATEGORY V1'].replace(('Traffic Control Device Improper/Non-Working', 'Pavement Slippery', 
                                        'Pavement Defective', 'Shoulders Defective/Improper', 
                                        'Lane Marking Improper/Inadequate'), ('Road Conditions'), inplace=True)

df['CONTRIBUTING CATEGORY V1'].replace(('Driverless/Runaway Vehicle', 'Other Vehicular', 'Oversized Vehicle'), 
                                       ('Third-Party (Vehicular)'), inplace=True)

df['CONTRIBUTING CATEGORY V1'].replace(('Animals Action', 
                                        'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion'), 
                                       ('Third-Party (Non-Vehicular)'), inplace=True)

df['CONTRIBUTING CATEGORY V1'].replace(('Passenger Distraction', 'Outside Car Distraction'), 
                                       ('Other Distractions'), inplace=True)

In [40]:
#Sanity check to confirm only 10 categories
df['CONTRIBUTING CATEGORY V1'].value_counts()

Moving Violation               108257
Driver Error                    86398
Third-Party (Vehicular)          8406
Bodily Impairment                6618
Environmental Factors            3565
Road Conditions                  2507
Vehicle Defects                  2399
Third-Party (Non-Vehicular)      2056
Other Distractions               1281
Internal Electronics Usage        171
Name: CONTRIBUTING CATEGORY V1, dtype: int64

In [41]:
#Convert columns from float to integers
df = df.astype({'ZIP CODE':'int64', 'NUMBER OF PERSONS INJURED':'int64', 'NUMBER OF PERSONS KILLED':'int64'})

In [42]:
#Drop remaining NaN values
df = df.dropna()

#### Season

In [43]:
#Create Season column
## Months 3-5 = Spring
## Months 6-8 = Summer
## Months 9-11 = Autumn
## Months 12-2 = Winter

df['SEASON'] = np.where(((df.index.month >= 3) & (df.index.month <= 5)), 'Spring',
                          np.where(((df.index.month >= 6) & (df.index.month <= 8)), 'Summer',
                                  np.where(((df.index.month >= 9) & (df.index.month <= 11)), 'Autumn', 'Winter')))

df['SEASON'].value_counts()

Winter    56663
Summer    55723
Autumn    54181
Spring    53546
Name: SEASON, dtype: int64

#### Time of Day

In [44]:
#Define the bins
bins=[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 24]

#Custom labels for time of day/hour intervals
labels=['Late Night', 'Early Morning', 'Dawn', 'Early AM', 'Morning', 'Late Morning', 'Early Afternoon', 
        'Afternoon', 'Evening', 'Night']

#Add the bins to the dataframe
df['TIME OF DAY'] = pd.cut(df.index.hour, bins, labels=labels, right=False)

#### Is Rush Hour

In [45]:
#Create and join dataframes indicating rush hour traffic
rush_hour = pd.concat([df.between_time('6:00', '9:00'), df.between_time('16:00', '19:00')])

#Create new column indicating if rush hour -- return True if index is in rush_hour, False if not
df['IS RUSH HOUR'] = df.index.isin(rush_hour.index)

#Sanity check to confirm there are True/False values
df['IS RUSH HOUR'].value_counts()

False    147475
True      72638
Name: IS RUSH HOUR, dtype: int64

In [46]:
df

,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,CONTRIBUTING FACTOR VEHICLE 1,COLLISION_ID,VEHICLE TYPE CODE 1,CONTRIBUTING CATEGORY V1,SEASON,TIME OF DAY,IS RUSH HOUR
CRASH DATE TIME,,,,,,,,,,,,,
2018-01-01 00:00:00,QUEENS,11419,40.682970,-73.828240,0,0,Passing Too Closely,3819067,Sedan,Moving Violation,Winter,Late Night,False
2018-01-01 00:00:00,BROOKLYN,11239,40.653060,-73.882020,0,0,Driver Inattention/Distraction,3820851,Sedan,Driver Error,Winter,Late Night,False
2018-01-01 00:00:00,BROOKLYN,11230,40.624180,-73.970480,0,0,Driver Inattention/Distraction,3822296,Station Wagon/Sport Utility Vehicle,Driver Error,Winter,Late Night,False
2018-01-01 00:00:00,BROOKLYN,11234,40.628770,-73.918250,1,0,Failure to Yield Right-of-Way,3820945,Sedan,Moving Violation,Winter,Late Night,False
2018-01-01 00:00:00,BROOKLYN,11230,40.623220,-73.961020,0,0,Driver Inattention/Distraction,3821055,Taxi,Driver Error,Winter,Late Night,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-27 20:44:00,BROOKLYN,11238,40.670660,-73.957985,1,0,Driver Inattention/Distraction,4616538,Station Wagon/Sport Utility Vehicle,Driver Error,Spring,Night,False
2023-03-27 21:50:00,QUEENS,11385,40.709038,-73.898460,1,0,Failure to Yield Right-of-Way,4616419,Moped,Moving Violation,Spring,Night,False
2023-03-27 22:10:00,BROOKLYN,11218,40.640410,-73.969124,1,0,Driver Inattention/Distraction,4616379,Sedan,Driver Error,Spring,Night,False


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 220113 entries, 2018-01-01 00:00:00 to 2023-03-27 23:46:00
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   BOROUGH                        220113 non-null  object  
 1   ZIP CODE                       220113 non-null  int64   
 2   LATITUDE                       220113 non-null  float64 
 3   LONGITUDE                      220113 non-null  float64 
 4   NUMBER OF PERSONS INJURED      220113 non-null  int64   
 5   NUMBER OF PERSONS KILLED       220113 non-null  int64   
 6   CONTRIBUTING FACTOR VEHICLE 1  220113 non-null  object  
 7   COLLISION_ID                   220113 non-null  int64   
 8   VEHICLE TYPE CODE 1            220113 non-null  object  
 9   CONTRIBUTING CATEGORY V1       220113 non-null  object  
 10  SEASON                         220113 non-null  object  
 11  TIME OF DAY                    220113 non-nu

## Date Selection for Modeling

In [48]:
#Create dataframe with only 2023 data - to be considered future
df_2023 = df['2023':]
df_2023.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6151 entries, 2023-01-01 00:00:00 to 2023-03-27 23:46:00
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   BOROUGH                        6151 non-null   object  
 1   ZIP CODE                       6151 non-null   int64   
 2   LATITUDE                       6151 non-null   float64 
 3   LONGITUDE                      6151 non-null   float64 
 4   NUMBER OF PERSONS INJURED      6151 non-null   int64   
 5   NUMBER OF PERSONS KILLED       6151 non-null   int64   
 6   CONTRIBUTING FACTOR VEHICLE 1  6151 non-null   object  
 7   COLLISION_ID                   6151 non-null   int64   
 8   VEHICLE TYPE CODE 1            6151 non-null   object  
 9   CONTRIBUTING CATEGORY V1       6151 non-null   object  
 10  SEASON                         6151 non-null   object  
 11  TIME OF DAY                    6151 non-null   category
 12

In [49]:
#Export as a .csv as final cleaned 2023dataset
df_2023.to_csv('data/Final_Motor_Vehicle_Collisions_QuBr_2023_V1.csv')

In [50]:
#Filter to crashes between 2018-2023
df = df['2018-01-01':'2022-12-31']
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 213962 entries, 2018-01-01 00:00:00 to 2022-12-31 23:50:00
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   BOROUGH                        213962 non-null  object  
 1   ZIP CODE                       213962 non-null  int64   
 2   LATITUDE                       213962 non-null  float64 
 3   LONGITUDE                      213962 non-null  float64 
 4   NUMBER OF PERSONS INJURED      213962 non-null  int64   
 5   NUMBER OF PERSONS KILLED       213962 non-null  int64   
 6   CONTRIBUTING FACTOR VEHICLE 1  213962 non-null  object  
 7   COLLISION_ID                   213962 non-null  int64   
 8   VEHICLE TYPE CODE 1            213962 non-null  object  
 9   CONTRIBUTING CATEGORY V1       213962 non-null  object  
 10  SEASON                         213962 non-null  object  
 11  TIME OF DAY                    213962 non-nu

Export smaller dataset that can be pushed to GitHub.

In [51]:
#Export as a .csv as final cleaned dataset
df.to_csv('data/Final_Motor_Vehicle_Collisions_QuBr_2018_2023_V1.csv')